In [1]:
print('''
l='depth_2'
k='depth_1'
j='depth_0'
i='df_base'
h='parquet_files'
g='/kaggle/input/home-credit-credit-risk-model-stability'
X='category'
W='D'
V=str
R=None
Q='WEEK_NUM'
N='date_decision'
J='target'
H='case_id'
import sys
from pathlib import Path
import subprocess,os,gc
from glob import glob
import numpy as A,pandas as Y,polars as B
from datetime import datetime
import seaborn as y,matplotlib.pyplot as z,joblib as m,warnings as n
n.filterwarnings('ignore')
S=g
from sklearn.model_selection import TimeSeriesSplit,GroupKFold,StratifiedGroupKFold
from sklearn.base import BaseEstimator,RegressorMixin
from sklearn.metrics import roc_auc_score
import lightgbm as o
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
class O:
	def set_table_dtypes(A):
		for C in A.columns:
			if C in[H,Q,'num_group1','num_group2']:A=A.with_columns(B.col(C).cast(B.Int64))
			elif C in[N]:A=A.with_columns(B.col(C).cast(B.Date))
			elif C[-1]in('P','A'):A=A.with_columns(B.col(C).cast(B.Float64))
			elif C[-1]in('M',):A=A.with_columns(B.col(C).cast(B.String))
			elif C[-1]in(W,):A=A.with_columns(B.col(C).cast(B.Date))
		return A
	def handle_dates(A):
		for C in A.columns:
			if C[-1]in(W,):A=A.with_columns(B.col(C)-B.col(N));A=A.with_columns(B.col(C).dt.total_days())
		A=A.drop(N,'MONTH');return A
	def filter_cols(A):
		for C in A.columns:
			if(C not in[J,H,Q])&(A[C].dtype==B.String):
				D=A[C].n_unique()
				if(D==1)|(D>200):A=A.drop(C)
		return A
class I:
	def num_expr(A):C=[A for A in A.columns if A[-1]in('P','A')];D=[B.max(A).alias(f"max_{A}")for A in C];return D
	def date_expr(A):C=[A for A in A.columns if A[-1]in W];D=[B.max(A).alias(f"max_{A}")for A in C];return D
	def str_expr(A):C=[A for A in A.columns if A[-1]in('M',)];D=[B.max(A).alias(f"max_{A}")for A in C];return D
	def other_expr(A):C=[A for A in A.columns if A[-1]in('T','L')];D=[B.max(A).alias(f"max_{A}")for A in C];return D
	def count_expr(A):C=[A for A in A.columns if'num_group'in A];D=[B.max(A).alias(f"max_{A}")for A in C];return D
	def get_exprs(A):B=I.num_expr(A)+I.date_expr(A)+I.str_expr(A)+I.other_expr(A)+I.count_expr(A);return B
def C(path,depth=R):
	A=B.read_parquet(path);A=A.pipe(O.set_table_dtypes)
	if depth in[1,2]:A=A.group_by(H).agg(I.get_exprs(A))
	return A
def K(regex_path,depth=R):
	C=[]
	for D in glob(V(regex_path)):
		A=B.read_parquet(D);A=A.pipe(O.set_table_dtypes)
		if depth in[1,2]:A=A.group_by(H).agg(I.get_exprs(A))
		C.append(A)
	A=B.concat(C,how='vertical_relaxed');A=A.unique(subset=[H]);return A
def Z(df_base,depth_0,depth_1,depth_2):
	A=df_base;A=A.with_columns(month_decision=B.col(N).dt.month(),weekday_decision=B.col(N).dt.weekday())
	for(C,D)in enumerate(depth_0+depth_1+depth_2):A=A.join(D,how='left',on=H,suffix=f"_{C}")
	A=A.pipe(O.handle_dates);return A
def a(df_data,cat_cols=R):
	B=cat_cols;A=df_data;A=A.to_pandas()
	if B is R:B=list(A.select_dtypes('object').columns)
	A[B]=A[B].astype(X);return A,B
def T(df):
	B=df;G=B.memory_usage().sum()/1024**2
	for C in B.columns:
		F=B[C].dtype
		if V(F)==X:continue
		if F!=object:
			D=B[C].min();E=B[C].max()
			if V(F)[:3]=='int':
				if D>A.iinfo(A.int8).min and E<A.iinfo(A.int8).max:B[C]=B[C].astype(A.int8)
				elif D>A.iinfo(A.int16).min and E<A.iinfo(A.int16).max:B[C]=B[C].astype(A.int16)
				elif D>A.iinfo(A.int32).min and E<A.iinfo(A.int32).max:B[C]=B[C].astype(A.int32)
				elif D>A.iinfo(A.int64).min and E<A.iinfo(A.int64).max:B[C]=B[C].astype(A.int64)
			elif D>A.finfo(A.float16).min and E<A.finfo(A.float16).max:B[C]=B[C].astype(A.float16)
			elif D>A.finfo(A.float32).min and E<A.finfo(A.float32).max:B[C]=B[C].astype(A.float32)
			else:B[C]=B[C].astype(A.float64)
		else:continue
	H=B.memory_usage().sum()/1024**2;return B
S=Path(g)
E=S/h/'train'
F=S/h/'test'
L={i:C(E/'train_base.parquet'),j:[C(E/'train_static_cb_0.parquet'),K(E/'train_static_0_*.parquet')],k:[K(E/'train_applprev_1_*.parquet',1),C(E/'train_tax_registry_a_1.parquet',1),C(E/'train_tax_registry_b_1.parquet',1),C(E/'train_tax_registry_c_1.parquet',1),K(E/'train_credit_bureau_a_1_*.parquet',1),C(E/'train_credit_bureau_b_1.parquet',1),C(E/'train_other_1.parquet',1),C(E/'train_person_1.parquet',1),C(E/'train_deposit_1.parquet',1),C(E/'train_debitcard_1.parquet',1)],l:[C(E/'train_credit_bureau_b_2.parquet',2)]}
D=Z(**L)
del L
gc.collect()
D=D.pipe(O.filter_cols)
D,p=a(D)
D=T(D)
b=D.select_dtypes(exclude=X).columns
from itertools import combinations,permutations
c=D[b].isna()
P={}
for U in b:
	d=c[U].sum()
	try:P[d].append(U)
	except:P[d]=[U]
del c
A0=gc.collect()
def q(grps):
	A=[]
	for B in grps:
		C=0;E=B[0]
		for F in B:
			G=D[F].nunique()
			if G>C:C=G;E=F
		A.append(E)
	return A
def r(matrix,threshold=.8):
	D=matrix;H=D.corr();E=[];A=list(D.columns)
	while A:
		B=A.pop(0);F=[B];G=[B]
		for C in A:
			if H.loc[B,C]>=threshold:F.append(C);G.append(C)
		E.append(F);A=[A for A in A if A not in G]
	return E
M=[]
for(s,e)in P.items():
	if len(e)>1:t=P[s];u=r(D[t],threshold=.8);v=q(u);M=M+v
	else:M=M+e
D=D[M]
L={i:C(F/'test_base.parquet'),j:[C(F/'test_static_cb_0.parquet'),K(F/'test_static_0_*.parquet')],k:[K(F/'test_applprev_1_*.parquet',1),C(F/'test_tax_registry_a_1.parquet',1),C(F/'test_tax_registry_b_1.parquet',1),C(F/'test_tax_registry_c_1.parquet',1),K(F/'test_credit_bureau_a_1_*.parquet',1),C(F/'test_credit_bureau_b_1.parquet',1),C(F/'test_other_1.parquet',1),C(F/'test_person_1.parquet',1),C(F/'test_deposit_1.parquet',1),C(F/'test_debitcard_1.parquet',1)],l:[C(F/'test_credit_bureau_b_2.parquet',2)]}
G=Z(**L)
del L
gc.collect()
G=G.select([A for A in D.columns if A!=J])
G,p=a(G)
G=T(G)
gc.collect()
D[J]=0
G[J]=1
D=Y.concat([D,G])
D=T(D)
w=D[J]
D=D.drop(columns=[J,H,Q])
f=o.LGBMClassifier()
f.fit(D,w)
G=G.drop(columns=[Q,J])
G=G.set_index(H)
x=Y.Series(f.predict_proba(G)[:,1],index=G.index)
m.dump(x,'y.pkl')
''', file=open("y.py", "w"), end="")

In [2]:
%%capture
!python y.py
!rm y.py

In [3]:
import gc
import lightgbm as lgb
import numpy as np
import pandas as pd
import polars as pl
from tqdm import tqdm
import joblib
import warnings
import os
from itertools import combinations, permutations
from catboost import CatBoostClassifier, Pool
from glob import glob
from IPython.display import display
from pathlib import Path
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedGroupKFold
from typing import Any

warnings.filterwarnings("ignore")

ROOT = Path("/kaggle/input/home-credit-credit-risk-model-stability")
TRAIN_DIR = ROOT / "parquet_files" / "train"
TEST_DIR = ROOT / "parquet_files" / "test"

def reduce_memory_usage(df: pl.DataFrame, name) -> pl.DataFrame:
    int_types = [
        pl.Int8,
        pl.Int16,
        pl.Int32,
        pl.Int64,
        pl.UInt8,
        pl.UInt16,
        pl.UInt32,
        pl.UInt64,
    ]
    float_types = [pl.Float32, pl.Float64]

    for col in df.columns:
        col_type = df[col].dtype
        if col_type in int_types + float_types:
            c_min = df[col].min()
            c_max = df[col].max()

            if c_min is not None and c_max is not None:
                if col_type in int_types:
                    if c_min >= 0:
                        if (
                            c_min >= np.iinfo(np.uint8).min
                            and c_max <= np.iinfo(np.uint8).max
                        ):
                            df = df.with_columns(df[col].cast(pl.UInt8))
                        elif (
                            c_min >= np.iinfo(np.uint16).min
                            and c_max <= np.iinfo(np.uint16).max
                        ):
                            df = df.with_columns(df[col].cast(pl.UInt16))
                        elif (
                            c_min >= np.iinfo(np.uint32).min
                            and c_max <= np.iinfo(np.uint32).max
                        ):
                            df = df.with_columns(df[col].cast(pl.UInt32))
                        elif (
                            c_min >= np.iinfo(np.uint64).min
                            and c_max <= np.iinfo(np.uint64).max
                        ):
                            df = df.with_columns(df[col].cast(pl.UInt64))
                    else:
                        if (
                            c_min >= np.iinfo(np.int8).min
                            and c_max <= np.iinfo(np.int8).max
                        ):
                            df = df.with_columns(df[col].cast(pl.Int8))
                        elif (
                            c_min >= np.iinfo(np.int16).min
                            and c_max <= np.iinfo(np.int16).max
                        ):
                            df = df.with_columns(df[col].cast(pl.Int16))
                        elif (
                            c_min >= np.iinfo(np.int32).min
                            and c_max <= np.iinfo(np.int32).max
                        ):
                            df = df.with_columns(df[col].cast(pl.Int32))
                        elif (
                            c_min >= np.iinfo(np.int64).min
                            and c_max <= np.iinfo(np.int64).max
                        ):
                            df = df.with_columns(df[col].cast(pl.Int64))
                elif col_type in float_types:
                    if (
                        c_min > np.finfo(np.float32).min
                        and c_max < np.finfo(np.float32).max
                    ):
                        df = df.with_columns(df[col].cast(pl.Float32))
    return df

def to_pandas(
    df: pl.DataFrame, cat_cols: list[str] = None
) -> (pd.DataFrame, list[str]):
    df: pd.DataFrame = df.to_pandas()

    if cat_cols is None:
        cat_cols = list(df.select_dtypes("object").columns)

    df[cat_cols] = df[cat_cols].astype("str")

    return df, cat_cols


class Aggregator:
    @staticmethod
    def max_expr(df: pl.LazyFrame) -> list[pl.Series]:
        cols: list[str] = [
            col
            for col in df.columns
            if (col[-1] in ("P", "M", "A", "D", "T", "L")) or ("num_group" in col)
        ]

        expr_max: list[pl.Series] = [
            pl.col(col).max().alias(f"max_{col}") for col in cols
        ]

        return expr_max

    @staticmethod
    def mean_expr(df: pl.LazyFrame) -> list[pl.Series]:
        cols: list[str] = [col for col in df.columns if col.endswith(("P", "A", "D"))]

        expr_mean: list[pl.Series] = [
            pl.col(col).mean().alias(f"mean_{col}") for col in cols
        ]

        return expr_mean

    @staticmethod
    def var_expr(df: pl.LazyFrame) -> list[pl.Series]:
        cols: list[str] = [col for col in df.columns if col.endswith(("P", "A", "D"))]

        expr_mean: list[pl.Series] = [
            pl.col(col).var().alias(f"var_{col}") for col in cols
        ]

        return expr_mean

    @staticmethod
    def get_exprs(df: pl.LazyFrame) -> list[pl.Series]:
        exprs = (
            Aggregator.max_expr(df) + Aggregator.mean_expr(df) + Aggregator.var_expr(df)
        )

        return exprs


def change_dtypes(df):
    for col in df.columns:
        if col == "case_id":
            df = df.with_columns(pl.col(col).cast(pl.UInt32).alias(col))
        elif col in ["WEEK_NUM", "num_group1", "num_group2"]:
            df = df.with_columns(pl.col(col).cast(pl.UInt16).alias(col))
        elif col == "date_decision" or col[-1] == "D":
            df = df.with_columns(pl.col(col).cast(pl.Date).alias(col))
        elif col[-1] in ["P", "A"]:
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))
        elif col[-1] == "M":
            df = df.with_columns(pl.col(col).cast(pl.String))
    return df


def scan_files(glob_path: str, depth: int = None):
    chunks = []
    for path in glob(str(glob_path)):
        df = pl.read_parquet(path, low_memory=True, rechunk=True)
        df = df.pipe(change_dtypes)
        if depth in [1, 2]:
            df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
        chunks.append(df)

    df = pl.concat(chunks, how="vertical_relaxed")
    del chunks
    gc.collect()
    df = df.unique(subset=["case_id"])
    return df


def join_dataframes(df_base, depth_0, depth_1, depth_2):
    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")

    return df_base


def transform_cols(df: pl.DataFrame) -> pl.DataFrame:
    if "riskassesment_302T" in df.columns:
        if df["riskassesment_302T"].dtype == pl.Null:
            df = df.with_columns(
                [
                    pl.Series(
                        "riskassesment_302T_rng", df["riskassesment_302T"], pl.UInt8
                    ),
                    pl.Series(
                        "riskassesment_302T_mean", df["riskassesment_302T"], pl.UInt8
                    ),
                ]
            )
        else:
            pct_low: pl.Series = (
                df["riskassesment_302T"]
                .str.split(" - ")
                .apply(lambda x: x[0].replace("%", ""))
                .cast(pl.UInt8)
            )
            pct_high: pl.Series = (
                df["riskassesment_302T"]
                .str.split(" - ")
                .apply(lambda x: x[1].replace("%", ""))
                .cast(pl.UInt8)
            )

            diff: pl.Series = pct_high - pct_low
            avg: pl.Series = ((pct_low + pct_high) / 2).cast(pl.Float32)

            del pct_high, pct_low
            gc.collect()

            df = df.with_columns(
                [
                    diff.alias("riskassesment_302T_rng"),
                    avg.alias("riskassesment_302T_mean"),
                ]
            )
        df.drop("riskassesment_302T")
    return df


def handle_dates(df: pl.DataFrame) -> pl.DataFrame:
    for col in df.columns:
        if col.endswith("D"):
            df = df.with_columns(pl.col(col) - pl.col("date_decision"))
            df = df.with_columns(pl.col(col).dt.total_days().cast(pl.Int32))
    df = df.rename({"WEEK_NUM": "week_num"})
    return df.drop("date_decision", "MONTH")


def filter_cols(df: pl.DataFrame) -> pl.DataFrame:
    for col in df.columns:
        if col not in ["case_id", "year", "month", "week_num", "target"]:
            null_pct = df[col].is_null().mean()

            if null_pct > 0.95:
                df = df.drop(col)

    for col in df.columns:
        if (col not in ["case_id", "year", "month", "week_num", "target"]) and (df[col].dtype == pl.String):
            n_unique = df[col].n_unique()

            if (n_unique > 200) | (n_unique == 1):
                df = df.drop(col)

    return df

data_store: dict = {
    "df_base": scan_files(TRAIN_DIR / "train_base.parquet"),
    "depth_0": [
        scan_files(TRAIN_DIR / "train_static_cb_0.parquet"),
        scan_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
    "depth_1": [
        scan_files(TRAIN_DIR / "train_applprev_1_*.parquet", 1),
        scan_files(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
        scan_files(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
        scan_files(TRAIN_DIR / "train_tax_registry_c_1.parquet", 1),
        scan_files(TRAIN_DIR / "train_credit_bureau_a_1_*.parquet", 1),
        scan_files(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
        scan_files(TRAIN_DIR / "train_other_1.parquet", 1),
        scan_files(TRAIN_DIR / "train_person_1.parquet", 1),
        scan_files(TRAIN_DIR / "train_deposit_1.parquet", 1),
        scan_files(TRAIN_DIR / "train_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        scan_files(TRAIN_DIR / "train_credit_bureau_a_2_*.parquet", 2),
        scan_files(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
        scan_files(TRAIN_DIR / "train_applprev_2.parquet", 2),
        scan_files(TRAIN_DIR / "train_person_2.parquet", 2),
    ],
}


df_train = (
    join_dataframes(**data_store)
    .pipe(filter_cols)
    .pipe(transform_cols)
    .pipe(handle_dates)
    .pipe(reduce_memory_usage, "df_train")
).to_pandas()
    
del data_store
gc.collect()

cat_cols = list(df_train.select_dtypes("object").columns)
df_train = df_train.sort_values(by="case_id", ascending=True)
df_train_columns = [col for col in df_train.columns if col != "target"]

if pd.read_csv(ROOT / "sample_submission.csv").shape[0] == 10:
    df_train = df_train.iloc[:50000]

df_train = df_train[df_train["week_num"] > 62]
X = df_train.drop(columns=["target", "case_id", "week_num"])
y = df_train["target"]
week_num = df_train["week_num"]

del df_train
gc.collect()

lgb_params_1 = {
    "boosting_type": "gbdt",
    "colsample_bynode": 0.8,
    "colsample_bytree": 0.8,
    "device": 'gpu',
    "extra_trees": True,
    "learning_rate": 0.05,
    "l1_regularization": 0.1,
    "l2_regularization": 10,
    "max_depth": 20,
    "metric": "auc",
    "n_estimators": 2000,
    "num_leaves": 64,
    "objective": "binary",
    "random_state": 42,
    "verbose": 1,
}

lgb_params_2 = {
    "boosting_type": "gbdt",
    "colsample_bynode": 0.8,
    "colsample_bytree": 0.8,
    "device": 'gpu',
    "extra_trees": True,
    "learning_rate": 0.03,
    "l1_regularization": 0.1,
    "l2_regularization": 10,
    "max_depth": 16,
    "metric": "auc",
    "n_estimators": 2000,
    "num_leaves": 72,
    "objective": "binary",
    "random_state": 42,
    "verbose": 1,
}

X[cat_cols] = X[cat_cols].astype("str")

cv = StratifiedGroupKFold(n_splits=10, shuffle=False)
for iter_idx, (idx_train, idx_valid) in enumerate(cv.split(X, y, groups=week_num)):
    X_train, y_train, week_num_train = (
        X.iloc[idx_train],
        y.iloc[idx_train],
        week_num.iloc[idx_train],
    )
    X_valid, y_valid, week_num_valid = (
        X.iloc[idx_valid],
        y.iloc[idx_valid],
        week_num.iloc[idx_valid],
    )

    train_pool = Pool(
        data=X_train, label=y_train, cat_features=cat_cols, weight=week_num_train
    )
    val_pool = Pool(
        data=X_valid, label=y_valid, cat_features=cat_cols, weight=week_num_valid
    )

    cat_boost_clf = CatBoostClassifier(
        best_model_min_trees = 1200,
        boosting_type = "Plain",
        eval_metric = "AUC",
        iterations = 600 if pd.read_csv(ROOT / "sample_submission.csv").shape[0] == 10 else 6000,
        learning_rate = 0.05,
        l2_leaf_reg = 10,
        max_leaves = 64,
        random_seed = 42,
        task_type = "GPU",
        use_best_model = True,
        verbose = 1,
    )

    cat_boost_clf.fit(train_pool, eval_set=val_pool, verbose=False)

    joblib.dump(cat_boost_clf, f"cat_boost_fold_{iter_idx}.pkl")

    X_train[cat_cols] = X_train[cat_cols].astype("category")
    X_valid[cat_cols] = X_valid[cat_cols].astype("category")

    if iter_idx % 2 == 0:
        light_gbm_clf = lgb.LGBMClassifier(**lgb_params_1)
    else:
        light_gbm_clf = lgb.LGBMClassifier(**lgb_params_2)

    light_gbm_clf.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[lgb.log_evaluation(100), lgb.early_stopping(100)],
    )
    
    joblib.dump(light_gbm_clf, f"light_gbm_fold_{iter_idx}.pkl")

    del (
        X_train,
        y_train,
        X_valid,
        y_valid,
        week_num_train,
        week_num_valid,
        train_pool,
        val_pool,
        cat_boost_clf,
        light_gbm_clf,
    )
    gc.collect()

del week_num, X, y
gc.collect()

data_store: dict = {
    "df_base": scan_files(TEST_DIR / "test_base.parquet"),
    "depth_0": [
        scan_files(TEST_DIR / "test_static_cb_0.parquet"),
        scan_files(TEST_DIR / "test_static_0_*.parquet"),
    ],
    "depth_1": [
        scan_files(TEST_DIR / "test_applprev_1_*.parquet", 1),
        scan_files(TEST_DIR / "test_tax_registry_a_1.parquet", 1),
        scan_files(TEST_DIR / "test_tax_registry_b_1.parquet", 1),
        scan_files(TEST_DIR / "test_tax_registry_c_1.parquet", 1),
        scan_files(TEST_DIR / "test_credit_bureau_a_1_*.parquet", 1),
        scan_files(TEST_DIR / "test_credit_bureau_b_1.parquet", 1),
        scan_files(TEST_DIR / "test_other_1.parquet", 1),
        scan_files(TEST_DIR / "test_person_1.parquet", 1),
        scan_files(TEST_DIR / "test_deposit_1.parquet", 1),
        scan_files(TEST_DIR / "test_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        scan_files(TEST_DIR / "test_credit_bureau_a_2_*.parquet", 2),
        scan_files(TEST_DIR / "test_credit_bureau_b_2.parquet", 2),
        scan_files(TEST_DIR / "test_applprev_2.parquet", 2),
        scan_files(TEST_DIR / "test_person_2.parquet", 2),
    ],
}

df_test: pl.DataFrame = (
    join_dataframes(**data_store)
    .pipe(transform_cols)
    .pipe(handle_dates)
    .select(df_train_columns)
    .pipe(reduce_memory_usage, "df_test")
)

del data_store
gc.collect()

fitted_models_cat = []
for path in glob("cat_boost_*.pkl"):
    fitted_models_cat.append(joblib.load(path))


fitted_models_lgb = []
for path in glob("light_gbm_*.pkl"):
    fitted_models_lgb.append(joblib.load(path))

df_test, cat_cols = to_pandas(df_test, cat_cols)

X_test: pd.DataFrame = df_test.drop(columns=["week_num"]).set_index("case_id")

X_test[cat_cols] = X_test[cat_cols].astype("category")


class VotingModel(BaseEstimator, ClassifierMixin):
    def __init__(self, estimators: list[BaseEstimator]):
        super().__init__()
        self.estimators = estimators

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)

    def predict_proba(self, X):
        y_preds = [
            estimator.predict_proba(X)[:, 1]
            if hasattr(estimator, "predict_proba")
            else estimator.predict(X)
            for estimator in self.estimators
        ]
        return np.mean(y_preds, axis=0)

ensemble_model = VotingModel(fitted_models_cat + fitted_models_lgb)

del df_train_columns, fitted_models_cat, fitted_models_lgb
gc.collect()

y_pred: pd.Series = pd.Series(ensemble_model.predict_proba(X_test), index=X_test.index)

df_subm: pd.DataFrame = pd.read_csv(ROOT / "sample_submission.csv")
df_subm = df_subm.set_index("case_id")
df_subm["score"] = y_pred
y_train_test_pred = joblib.load("y.pkl")
condition = y_train_test_pred < 0.978
df_subm.loc[condition, "score"] = (df_subm.loc[condition, "score"] - 0.0718).clip(0)

display(df_subm)
df_subm.to_csv("submission.csv")

del X_test, y_pred, df_subm
gc.collect()

CatBoostError: Labels variable is empty.